In [5]:
import numpy as np
import os
import _pickle as cPickle
from matplotlib import pyplot as plt

class CifarLoader(object):
    def __init__(self, source_files):
        self._source = source_files
        self._i = 0
        self.images = None
        self.labels = None
        
    def load(self):
        data = [unpickle(f) for f in self._source]
        images = np.vstack([d["data"] for d in data])
        n = len(images)
        self.images = images.reshape(n, 3, 32, 32).transpose(0, 2, 3, 1).astype(float) / 255
        self.labels = one_hot(np.hstack([d["labels"] for d in data]), 10)
        return self
    
    def next_batch(self, batch_size):
        x, y = self.images[self._i:self._i+batch_size], self.labels[self._i:self._i+batch_size]
        self._i = (self._i + batch_size) % len(self.images)
        return x, y

DATA_PATH = "CIFAR10/cifar10batchespy"

def unpickle(file):
    with open(os.path.join(DATA_PATH, file), 'rb') as fo:
        dict = cPickle.load(fo, encoding='latin1')
        return dict
def one_hot(vec, vals=10):
    n = len(vec)
    out = np.zeros((n, vals))
    out[range(n), vec] = 1
    return out

class CifarDataManager(object):
    def __init__(self):
        self.train = CifarLoader(["data_batch_{}".format(i) for i in range(1, 6)]).load()
        self.test = CifarLoader(["test_batch"]).load()

d = CifarDataManager()

images = d.train.images
labels = d.train.labels

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D

# Create the model
model = Sequential()
model.add(Conv2D(16, (5, 5), input_shape=(32, 32, 3), padding='same', activation='relu'))
model.add(Dropout(0.2))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))
# Compile model
epochs = 10
lrate = 0.1
sgd = SGD(lr=lrate)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# Fit the model
model.fit(images, labels, epochs=epochs, batch_size=128)
# Final evaluation of the model
scores = model.evaluate(d.test.images, d.test.labels)
print(scores)

Epoch 1/10
50000/50000 [==============================] - 268s - loss: 2.1494 - acc: 0.2110   
Epoch 2/10
50000/50000 [==============================] - 289s - loss: 1.8258 - acc: 0.3420   
Epoch 3/10
50000/50000 [==============================] - 297s - loss: 1.5533 - acc: 0.4377   
Epoch 4/10
50000/50000 [==============================] - 296s - loss: 1.4115 - acc: 0.4926   
Epoch 5/10
50000/50000 [==============================] - 279s - loss: 1.3009 - acc: 0.5328   
Epoch 6/10
50000/50000 [==============================] - 279s - loss: 1.2028 - acc: 0.5727   
Epoch 7/10
50000/50000 [==============================] - 290s - loss: 1.1073 - acc: 0.6097   
Epoch 8/10
50000/50000 [==============================] - 292s - loss: 1.0073 - acc: 0.6450   
Epoch 9/10
50000/50000 [==============================] - 262s - loss: 0.9133 - acc: 0.6772   
Epoch 10/10
 9984/10000 [============================>.] - ETA: 0s[0.9912084984779358, 0.65129999999999999]
